## Implementation of 1D Kalman filter

### The initial code implements the measurement and update functions of 1D Kalman Filters. 
---
1. For the measurements:
Given an initial distibution $A(\mu,\sigma^2)$ and a measured distribution $B(v,r^2)$, the updated mean is given as: $$\mu' = \frac{r^2 \mu + \sigma^2 v}{r^2 + \sigma^2}$$ and the updated variance is given as: $$\sigma^{2'} = \frac{1}{\frac{1}{r^2} + \frac{1}{\sigma^2}}$$

2. FOr the motion:
Given the initial distribution of the measurement $A$ and a measured motion $B$, the updated mean is given as: $$ \mu' = \mu + u$$ and the updated variance is given as: $$\sigma^{2'} = \sigma^2 + r^2$$.
---
The python implementation of these steps are given below

In [3]:
# Write a program that will iteratively update and
# predict based on the location measurements 
# and inferred motions shown below. 

#Function for measurement updates
def update(mean1, var1, mean2, var2):
    new_mean = float(var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

#Function for prediction updates
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

#Initial sample data
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

#Please print out ONLY the final values of the mean
#and the variance in a list [mu, sig]. 

#Python code for 1D Kalman Filter
for i in range(len(measurements)):
    mu, sig = update(mu, sig, measurements[i], measurement_sig)
    mu, sig = predict(mu,sig, motion[i], motion_sig)


print(mu, sig)


10.999906177177365 4.005861580844194


Having completed a 1D Kalman filter, next we write code for a multidimensional Kalman filter. For an initial estimate $X$, an uncertianty covariance $P$, a state transition matrix $F$, a motion vector $U$, movement estimate $Z$, measurement function $H$ and measurement noise $R$ the update is computed using the following equations. First the error: $$Y = Z-HX$$. Then we compute $S$ which projects the uncertainty into the measurement space $H$ and adds noise $R$ $$S = HPH^T + R$$ $$K = PH^TS^{-1}$$ K is the Kalman gain and then we compute the updates $X'$ and $P'$ $$X' = X + KY$$ $$P' = (I - KH)P$$. Using the updates Kalman filter makes predictions as $$X' = FX +U $$ and $$P' = FPF^T $$

### Example from Udacity below

In [19]:
# Write a function 'kalman_filter' that implements a multi-
# dimensional Kalman Filter for the example given

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError("Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError("Invalid size of matrix" )
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError("Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError("Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError("Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise ValueError("Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise ValueError("Zero diagonal")
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)



In [72]:
# Implementation of the filter below

def kalman_filter(x, P):
    for n in range(len(measurements)):        
        # measurement update
        y = matrix([[measurements[n]]]) - H*x  #y computes the error
        S = H*P*H.transpose() + R #S projects the uncertainty into the measurement space H and adds noise R
        k = P*H.transpose()*S.inverse() #K is the Kalman gain
        x = x + k*y #update of estimate
        P = (I - k*H)*P #update of uncertainty
        # prediction
        x = F*x + u
        P = F*P*F.transpose()
    return x,P

In [75]:
############################################
### use the code below to test your filter!
############################################



measurements = [1, 2, 3]
x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix


x1,P1 = kalman_filter(x, P)
print(x1)
print(P1)
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]

[[3.9996664447958645], [0.9999998335552873]]
[[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]
